Load libraries

In [17]:
require(ggplot2)
#require(GGally)
require(reshape2)
require(lme4)
require(compiler)
require(parallel)
require(boot)
# require(aod)
require(dplyr)

# Load data and set factors

In [37]:
mydata <- read.csv("C:\\Users\\sperez8\\Documents\\Personal Content\\Lab_study_data\\all_massaged_data\\dataframe_all_factors_for_analysis.txt",sep = '\t')
# # SID is the student number
mydata$sid <- factor(mydata$sid)
mydata$sim_index <- factor(mydata$sim_index)
mydata$lab_experience <- factor(mydata$lab_experience)
mydata$similar_sim <- factor(mydata$similar_sim)
mydata$pre_statement <- factor(mydata$pre_statement)
mydata$pre_correct <- factor(mydata$pre_correct)
mydata$similar_sim <- factor(mydata$similar_sim)
# mydata$CVS_context <- factor(mydata$CVS_context)

In [38]:
head(mydata)

sid,sim,variable,model,pre_statement,pre_correct,main_statement,main_correct,CVS_context,qual_score,...,use_area,use_separation,activity_order,sim_index,similar_sim,lab_experience,prior_number_virtual_labs,overall.POCC,PC1,PC2
10127163,C,Area,ident,0,NA,0,NA,2,1,...,1,1,LC,2,0,1,1,17,-0.5087537,0.1523806
10127163,C,Area,qual,1,1,1,1,2,1,...,1,1,LC,2,0,1,1,17,-0.5087537,0.1523806
10127163,C,Area,quant,1,1,1,1,2,1,...,1,1,LC,2,0,1,1,17,-0.5087537,0.1523806
10127163,C,Separation,ident,0,NA,0,NA,2,1,...,1,1,LC,2,0,1,1,17,-0.5087537,0.1523806
10127163,C,Separation,qual,1,1,1,1,2,1,...,1,1,LC,2,0,1,1,17,-0.5087537,0.1523806
10127163,C,Separation,quant,1,1,1,1,2,1,...,1,1,LC,2,0,1,1,17,-0.5087537,0.1523806


## Model
Model is:

main_correct ~  CVS_context*model + variable + pre_statement + pre_correct + sim_index + lab_experience + similar_sim + prior_number_virtual_labs

with sid as the student id (8 digit number)

All the variables are binary or categorical except for "CVS_context" and "prior_number_virtual_labs"

https://rstudio-pubs-static.s3.amazonaws.com/33653_57fc7b8e5d484c909b615d8633c01d51.html

Mixed effects with sid as random effects

In [56]:
lm_data = mydata[mydata$model == "quant", ]

mixed <- glmer(
    main_correct
    ~  CVS_context + variable + sim_index + pre_correct
    + lab_experience + similar_sim + prior_number_virtual_labs + (1 | sid),
           data = lm_data[complete.cases(lm_data), ], family = binomial, 
           control = glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)), nAGQ = 10)

summary(mixed)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"unable to evaluate scaled gradient"Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
" Hessian is numerically singular: parameters are not uniquely determined"Warning message in vcov.merMod(object, use.hessian = use.hessian):
"variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX"Warning message in vcov.merMod(object, correlation = correlation, sigm = sig):
"variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX"

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: main_correct ~ CVS_context + variable + sim_index + pre_correct +  
    lab_experience + similar_sim + prior_number_virtual_labs +      (1 | sid)
   Data: lm_data[complete.cases(lm_data), ]
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

     AIC      BIC   logLik deviance df.resid 
    35.6     58.3     -6.8     13.6       47 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.42017  0.00000  0.00002  0.02377  0.85081 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 3.253    1.804   
Number of obs: 58, groups:  sid, 29

Fixed effects:
                           Estimate Std. Error z value Pr(>|z|)
(Intercept)                   1.750  88067.515   0.000    1.000
CVS_context                  -0.772      1.365  -0.566    0.572
variableConcentration        14.930  

In [57]:
numcols <- grep("^c\\.",names(mydata))
mydata_sc <- mydata
mydata_sc[,numcols] <- scale(mydata_sc[,numcols])
mixed_sc <- update(mixed,data=mydata_sc)
summary(mixed_sc)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model is nearly unidentifiable: large eigenvalue ratio
 - Rescale variables?"

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: main_correct ~ CVS_context + variable + sim_index + pre_correct +  
    lab_experience + similar_sim + prior_number_virtual_labs +      (1 | sid)
   Data: mydata_sc
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

     AIC      BIC   logLik deviance df.resid 
   118.8    159.9    -48.4     96.8      300 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.66757  0.01725  0.03975  0.07208  0.74085 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 10.45    3.233   
Number of obs: 311, groups:  sid, 112

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)  
(Intercept)                21.6778   387.0381   0.056   0.9553  
CVS_context                 0.1906     0.6040   0.316   0.7523  
variableConcentration       0.1551     1.5454   0.100   0

In [58]:
tt <- getME(mixed_sc,"theta")
ll <- getME(mixed_sc,"lower")
min(tt[ll==0])

[1] 3.233004

In [59]:
derivs1 <- mixed_sc@optinfo$derivs
sc_grad1 <- with(derivs1,solve(Hessian,gradient))
max(abs(sc_grad1))

[1] 0.1259527

In [60]:
max(pmin(abs(sc_grad1),abs(derivs1$gradient)))

[1] 1.979596e-06

In [61]:
ss <- getME(mixed_sc,c("theta","fixef"))
m2 <- update(mixed_sc,start=ss,control=glmerControl(optCtrl=list(maxfun=2e4)))

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.330368 (tol = 0.001, component 1)"Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model is nearly unidentifiable: large eigenvalue ratio
 - Rescale variables?"

In [62]:
m3 <- update(mixed_sc,start=ss,control=glmerControl(optimizer="bobyqa",
                            optCtrl=list(maxfun=2e5)))

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model is nearly unidentifiable: large eigenvalue ratio
 - Rescale variables?"

## other tstuff

A bunch of relevels

In [ ]:
mydata$variable <- relevel(mydata$variable, "Concentration")

mixed <- glmer(
    CVS_graph_3 
    ~ pre + variable + sim_index +level_experience_sims 
    + experience_undergrad_labs + used_similar_sim + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

In [ ]:
mydata$variable <- relevel(mydata$variable, "Separation")

mixed <- glmer(
    CVS_graph_3 
    ~ pre + variable + sim_index +level_experience_sims 
    + experience_undergrad_labs + used_similar_sim + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

In [ ]:
mydata$variable <- relevel(mydata$variable, "Width")

mixed <- glmer(
    CVS_graph_3 
    ~ pre + variable + sim_index +level_experience_sims 
    + experience_undergrad_labs + used_similar_sim + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)